## GaussianNB and Missing values 

In [ ]:
# the usual setup: 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# load training data
df = pd.read_csv('data/atlas-higgs-challenge-2014-v2.csv.gz')
# map y values to integers
df['Label'] = df['Label'].map({'b':0, 's':1});

In [ ]:
# let's create separate arrays for ML models
eventID = df['EventId']
X = df.loc[:,'DER_mass_MMC':'PRI_jet_all_pt']
y = df['Label']
weight = df['Weight']

In [ ]:
# now split data and weights into testing and training samples
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test, eventID_train, event_ID_test, weight_train, weight_test = train_test_split(
    X, y, eventID, weight, test_size=0.33, random_state=42)

### GaussianNB

Naive Bayes classifiers are built on Bayesian classification methods.
Use Bayes’s theorem:
$$P(B|A)P(A) = P(A|B)P(B)$$
to write
$$
P(L|\text{features}) = \frac{ P(\text{features}|L) P(L) }{ P(\text{features}) }
$$
where "L" is the label.

From this the ratio of the probability of two labels of a sample given the features can be computed as:
$$
\frac{ P(L_1|\text{features}) }{ P(L_2|\text{features}) } = \frac{ P(\text{features}|L_1)P(L_1) }{ P(\text{features}|L_2)P(L_2) }
$$
(Note that $P(\text{features})$, i.e. the probability distribution of the features, cancels out.)

In the Gaussian Naive Bayes classifier, the assumption is that data from each label is drawn from a simple Gaussian distribution.

In [ ]:
# GaussianNB (Gaussian Naive Bayes, assumes each class is drawn from an axis-aligned Gaussian distribution)
from sklearn.naive_bayes import GaussianNB # 1. choose model class
model = GaussianNB()                       # 2. instantiate model
model.fit(X_train, y_train);               # 3. fit model to data

In [ ]:
model.score(X_test, y_test)

In [ ]:
# define function to plot probability for sig/bg
def plot_proba( df, model, x ):
    df['Prob']=model.predict_proba(x)[:, 1]
    kwargs = dict(histtype='stepfilled', alpha=0.3, bins=40, density=True)
    df[df.Label==0].Prob.hist(label='Background',**kwargs)
    df[df.Label==1].Prob.hist(label='Signal',**kwargs)
    _=plt.legend()

In [ ]:
plot_proba(df, model, X)
print(model.score(X_test, y_test))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, model.predict(X_test))

In [ ]:
modelpars0 = pd.DataFrame({"mean": model.theta_[0,:], "sigma": np.sqrt(model.sigma_[0,:])}, index = X.keys())
modelpars1 = pd.DataFrame({"mean": model.theta_[1,:], "sigma": np.sqrt(model.sigma_[1,:])}, index = X.keys())

We can visualize the fits done by GaussianNB:

In [ ]:
import scipy.stats as stats

def plot(featurepars, data, color):
    xmin = featurepars["mean"] - 3*featurepars["sigma"]
    xmax = featurepars["mean"] + 3*featurepars["sigma"]
    ax.hist(data, bins = 100, range = (xmin, xmax), color = color, alpha = 0.3);
    x = np.linspace(xmin, xmax, 100)
    ax.plot(x, 
            stats.norm.pdf(x, featurepars["mean"], featurepars["sigma"])*len(data)/100*(xmax - xmin), 
            color = color)
    
fig, axes = plt.subplots(5, 5, figsize=(18, 16))
for idx, ax in enumerate(axes.ravel()):
    var = X.keys()[idx]
    ax.set_xlabel(var)
    # plot signal
    plot(modelpars1.loc[var], X[y == 1][var], "g")
    # plot background
    plot(modelpars0.loc[var], X[y == 0][var], "r")


### Replace missing values

In [ ]:
# (re-)compute cross-validation score on original model
from sklearn.model_selection import cross_val_score
cross_val_score(GaussianNB(), X, y).mean()

In [ ]:
X2 = X.where(X != -999, 0) # -999 -> 0
# retrain model and determine score (should slightly improve)
print("Strategy '%s' yields a score of %.4f" % ("constant", cross_val_score(GaussianNB(), X2, y).mean()))

In [ ]:
# replace placeholder values (-999.)
#X = X.where(X != -999, np.nan) # -999 -> NaN
from sklearn.preprocessing import Imputer
for strategy in ["mean", "median", "most_frequent"]:
    imp = Imputer(missing_values = -999, strategy = strategy)
    X2 = imp.fit_transform(X)
    # retrain model and determine score (should slightly improve)
    print("Strategy '%s' yields a score of %.4f" % (strategy, cross_val_score(GaussianNB(), X2, y).mean()))